In [1]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
import pytz


# Start the timer
start_time = time.time()

base_url = 'https://www.immobiliare.it/vendita-case/trieste-provincia/?criterio=rilevanza&pag={}'

def get_individual(url):
    req = requests.get(url).text
    soup = BeautifulSoup(req, 'html.parser')

    title = soup.find('div', class_='in-titleBlock__content').find('h1').text if soup.find('div', class_='in-titleBlock__content') else None
    title_text = title.split(',', 1) if title else [None]
    titolo = title_text[0].strip()
    area = title_text[1].strip() if len(title_text) > 1 else None


    ul_tag = soup.find('ul', class_='nd-list nd-list--pipe in-feat in-feat--full in-feat__mainProperty in-landingDetail__mainFeatures')
    price_li = ul_tag.find('li', class_='nd-list__item in-feat__item in-feat__item--main in-detail__mainFeaturesPrice')
    locali_li = ul_tag.find(lambda tag: tag.name == 'li' and tag.get('aria-label') in ['locali', 'locale'])
    superficie_li = ul_tag.find('li', attrs={'aria-label': 'superficie'})
    bagni_li = ul_tag.find(lambda tag: tag.name == 'li' and tag.get('aria-label') in ['bagni', 'bagno'])
    piano_li = ul_tag.find('li', attrs={'aria-label': 'piano'})


    if piano_li is None:
        dl_tag = soup.find('dl', class_='in-realEstateFeatures__list')
        dt_tags = dl_tag.find_all('dt', class_='in-realEstateFeatures__title')
        dd_tags = dl_tag.find_all('dd', class_='in-realEstateFeatures__value')
        for dt, dd in zip(dt_tags, dd_tags):
            if dt.text.strip() == 'piano':
                piano = dd.text.strip()
                break
        else:
            piano = None
    else:
        piano = piano_li.text.strip()

    price = price_li.text.strip() if price_li else None
    locali = locali_li.text.strip() if locali_li else None
    superficie = superficie_li.text.strip() if superficie_li else None
    bagni = bagni_li.text.strip() if bagni_li else None


    # Extract description
    dettagli_tag = soup.find('div', class_='in-readAll')
    dettagli = dettagli_tag.text.strip() if dettagli_tag else None
    descrizione = dettagli.split('.')[0].strip() if dettagli else None

    div_tag = soup.find('div', class_='in-referent in-referent__withPhone') or soup.find('div', class_='in-referent')
    agenzia = div_tag.find('p').text.strip() if div_tag and div_tag.find('p') else None

    DB_features = {}
    dl_tags = soup.find_all('dl', class_='in-realEstateFeatures__list')
    for dl_tag in dl_tags:
        if dl_tag:
            dt_tags = dl_tag.find_all('dt', class_='in-realEstateFeatures__title')
            dd_tags = dl_tag.find_all('dd', class_='in-realEstateFeatures__value')
            for dt, dd in zip(dt_tags, dd_tags):
                dt_text = dt.text.strip().lower()
                DB_features[dt_text] = dd.text.strip()

    # Filtering out unwanted keys from altre_caratteristiche
    unwanted_keys = ['superficie', 'locali', 'prezzo', 'spese condominio', 'cauzione','piano']
    DB_features = {k: v for k, v in DB_features.items() if k not in unwanted_keys}

    #for the blueprint url
    bp_tag = soup.find('div', class_='nd-figure__image nd-ratio nd-ratio--wide in-landingDetail__simpleGallery')
    img_tag = bp_tag.find('img', class_='nd-figure__content nd-ratio__img') if bp_tag else None

    # One-liner if-else statement
    DB_features['Blueprint Url'] = img_tag.get('src', 'Not available') if img_tag else 'Not available'
    
    
    # Include the date and time the property was extracted
    extracted_datetime = datetime.now()
    
    # Get the GMT time zone
    gmt = pytz.timezone('GMT')
    
    # Convert the extracted_datetime to GMT time zone
    extracted_datetime_gmt = extracted_datetime.astimezone(gmt)

    all_features = {}



    GS_features = {
        'Date_GMT': extracted_datetime_gmt.strftime('%Y-%m-%d %H:%M:%S %Z'),
        'Titolo': titolo,
        'Area':area,
        'Link': url,
        'Prezzo': price,
        'Locali': locali,
        'MQ': superficie,
        'Bagni': bagni,
        'Piano':piano,
        'Descrizione': descrizione,
        'Dettagli': dettagli,
        'Agenzia': agenzia,

    }

    all_features.update(GS_features)
    all_features.update(DB_features)

    return all_features

def scrape_data():
    total_properties = 0
    total_pages = 0

    url = base_url.format(1)
    req = requests.get(url).text
    soup = BeautifulSoup(req, 'html.parser')

    total_properties = int(soup.find('div', class_='in-searchList__title').text.split()[0].replace('.', ''))
    total_pages = (total_properties + 24) // 25

    all_urls = []
    for page_number in range(1, total_pages + 1):
        url = base_url.format(page_number)
        req = requests.get(url).text
        soup = BeautifulSoup(req, 'html.parser')
        house_list = soup.find_all('div', class_='nd-mediaObject__content in-card__content in-realEstateListCard__content')

        for house in house_list:
            house_url = house.find('a')['href']
            all_urls.append(house_url)

    print(all_urls)

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(get_individual, url) for url in all_urls]

    details_list = [future.result() for future in futures]
    df = pd.DataFrame(details_list)

    return df



# Scrape data and print the resulting DataFrame
dataframe = scrape_data()

#Getting the dataframe for the Google sheet
GS_columns = ['Date_GMT', 'Titolo', 'Area', 'Link', 'Prezzo', 'Locali', 'MQ', 'Bagni','Piano', 'Descrizione', 'Dettagli', 'Agenzia']
GS_df = dataframe[GS_columns]

#Geting the dataframe for the Database
DB_columns = ['Date_GMT', 'Titolo', 'Area', 'Link', 'Prezzo', 'Locali', 'MQ', 'Bagni','Piano', 'Descrizione', 'Dettagli', 'Agenzia', 
              'contratto', 'anno di costruzione', 'stato', 'riscaldamento','efficienza energetica', 'altre caratteristiche','Blueprint Url']

DB_df = dataframe[DB_columns]


# Calculate the elapsed time
elapsed_time = time.time() - start_time

# Print the elapsed time
print(f"Elapsed time: {elapsed_time} seconds")

['https://www.immobiliare.it/annunci/104849015/', 'https://www.immobiliare.it/annunci/98582788/', 'https://www.immobiliare.it/annunci/99036760/', 'https://www.immobiliare.it/annunci/85478590/', 'https://www.immobiliare.it/annunci/104926247/', 'https://www.immobiliare.it/annunci/104926007/', 'https://www.immobiliare.it/annunci/100489932/', 'https://www.immobiliare.it/annunci/101318745/', 'https://www.immobiliare.it/annunci/104477685/', 'https://www.immobiliare.it/annunci/103296092/', 'https://www.immobiliare.it/annunci/104067959/', 'https://www.immobiliare.it/annunci/102963330/', 'https://www.immobiliare.it/annunci/102815942/', 'https://www.immobiliare.it/annunci/101723507/', 'https://www.immobiliare.it/annunci/102038944/', 'https://www.immobiliare.it/annunci/103828865/', 'https://www.immobiliare.it/annunci/103449562/', 'https://www.immobiliare.it/annunci/103618852/', 'https://www.immobiliare.it/annunci/104561329/', 'https://www.immobiliare.it/annunci/104847719/', 'https://www.immobilia

In [2]:
dataframe.head()

,Date_GMT,Titolo,Area,Link,Prezzo,Locali,MQ,Bagni,Piano,Descrizione,...,numero immobili,categoria,aggiornato il,numero procedura,procedura,tribunale,referente,termine presentazione,giudice,curatore
0,2023-07-31 17:13:44 GMT,Trilocale viale Miramare 23,"Roiano, Trieste",https://www.immobiliare.it/annunci/104849015/,€ 270.000,3,135m²,1,3,PDM Real Estate propone in vendita questo spet...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-07-31 17:13:45 GMT,Bilocale Strada di Portopiccolo,Duino-Aurisina,https://www.immobiliare.it/annunci/98582788/,€ 489.000,2,84m²,1,1,NUOVO APPARTAMENTO - Nel prestigioso complesso...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-07-31 17:13:45 GMT,Appartamenti e Attici di nuova costruzione a T...,None,https://www.immobiliare.it/annunci/99036760/,€ 160.000 - € 500.000,1 - 4,da57m²,None,None,Le Terrazze sul Golfo è il nuovo progetto resi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-07-31 17:13:44 GMT,Attico viale Miramare 109,"Barcola, Trieste",https://www.immobiliare.it/annunci/85478590/,€ 650.000,3,105m²,2,"2 piani: Seminterrato, 4°, con ascensore, con ...",Meraviglioso attico nel residence nel compless...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-07-31 17:13:44 GMT,Trilocale viale Miramare 23,"Roiano, Trieste",https://www.immobiliare.it/annunci/104926247/,€ 270.000,3,135m²,1,3,Proponiamo in vendita questo spettacolare appa...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
GS_df.head()

,Date_GMT,Titolo,Area,Link,Prezzo,Locali,MQ,Bagni,Piano,Descrizione,Dettagli,Agenzia
0,2023-07-31 17:13:44 GMT,Trilocale viale Miramare 23,"Roiano, Trieste",https://www.immobiliare.it/annunci/104849015/,€ 270.000,3,135m²,1,3,PDM Real Estate propone in vendita questo spet...,PDM Real Estate propone in vendita questo spet...,SPUNTON & PARTNERS
1,2023-07-31 17:13:45 GMT,Bilocale Strada di Portopiccolo,Duino-Aurisina,https://www.immobiliare.it/annunci/98582788/,€ 489.000,2,84m²,1,1,NUOVO APPARTAMENTO - Nel prestigioso complesso...,NUOVO APPARTAMENTO - Nel prestigioso complesso...,PORTO PICCOLO
2,2023-07-31 17:13:45 GMT,Appartamenti e Attici di nuova costruzione a T...,None,https://www.immobiliare.it/annunci/99036760/,€ 160.000 - € 500.000,1 - 4,da57m²,None,None,Le Terrazze sul Golfo è il nuovo progetto resi...,Le Terrazze sul Golfo è il nuovo progetto resi...,Terrazze sul Golfo
3,2023-07-31 17:13:44 GMT,Attico viale Miramare 109,"Barcola, Trieste",https://www.immobiliare.it/annunci/85478590/,€ 650.000,3,105m²,2,"2 piani: Seminterrato, 4°, con ascensore, con ...",Meraviglioso attico nel residence nel compless...,Meraviglioso attico nel residence nel compless...,Gruppo Greif
4,2023-07-31 17:13:44 GMT,Trilocale viale Miramare 23,"Roiano, Trieste",https://www.immobiliare.it/annunci/104926247/,€ 270.000,3,135m²,1,3,Proponiamo in vendita questo spettacolare appa...,Proponiamo in vendita questo spettacolare appa...,STUDIO ROIANO S.R.L.


In [4]:
DB_df.head()

,Date_GMT,Titolo,Area,Link,Prezzo,Locali,MQ,Bagni,Piano,Descrizione,Dettagli,Agenzia,contratto,anno di costruzione,stato,riscaldamento,efficienza energetica,altre caratteristiche,Blueprint Url
0,2023-07-31 17:13:44 GMT,Trilocale viale Miramare 23,"Roiano, Trieste",https://www.immobiliare.it/annunci/104849015/,€ 270.000,3,135m²,1,3,PDM Real Estate propone in vendita questo spet...,PDM Real Estate propone in vendita questo spet...,SPUNTON & PARTNERS,Vendita,1905,Da ristrutturare,"Autonomo, a radiatori, alimentato a metano",F109.04 kWh/m² anno,Esposizione esternaBalconeImpianto tv singoloP...,https://pic.im-cdn.it/plan/99318099/m.jpg
1,2023-07-31 17:13:45 GMT,Bilocale Strada di Portopiccolo,Duino-Aurisina,https://www.immobiliare.it/annunci/98582788/,€ 489.000,2,84m²,1,1,NUOVO APPARTAMENTO - Nel prestigioso complesso...,NUOVO APPARTAMENTO - Nel prestigioso complesso...,PORTO PICCOLO,Vendita - Scarica capitolato,2014,Nuovo / In costruzione,"Centralizzato, a pavimento, alimentato a pompa...",A≥ 175 kWh/m² anno,Cancello elettricoFibra otticaPorta blindataEs...,https://pic.im-cdn.it/plan/99253501/m.jpg
2,2023-07-31 17:13:45 GMT,Appartamenti e Attici di nuova costruzione a T...,None,https://www.immobiliare.it/annunci/99036760/,€ 160.000 - € 500.000,1 - 4,da57m²,None,None,Le Terrazze sul Golfo è il nuovo progetto resi...,Le Terrazze sul Golfo è il nuovo progetto resi...,Terrazze sul Golfo,Vendita,NaN,NaN,"Centralizzato, a pavimento, alimentato a pompa...",NaN,Parcheggio biciGiardino comuneInfissi esterni ...,Not available
3,2023-07-31 17:13:44 GMT,Attico viale Miramare 109,"Barcola, Trieste",https://www.immobiliare.it/annunci/85478590/,€ 650.000,3,105m²,2,"2 piani: Seminterrato, 4°, con ascensore, con ...",Meraviglioso attico nel residence nel compless...,Meraviglioso attico nel residence nel compless...,Gruppo Greif,Vendita,2021,Nuovo / In costruzione,"Autonomo, a pavimento, alimentato a pompa di c...",A+≥ 175 kWh/m² anno,IdromassaggioCampo da tennisCancello elettrico...,Not available
4,2023-07-31 17:13:44 GMT,Trilocale viale Miramare 23,"Roiano, Trieste",https://www.immobiliare.it/annunci/104926247/,€ 270.000,3,135m²,1,3,Proponiamo in vendita questo spettacolare appa...,Proponiamo in vendita questo spettacolare appa...,STUDIO ROIANO S.R.L.,Vendita,1905,Da ristrutturare,"Autonomo, a radiatori, alimentato a gas",F109.04 kWh/m² anno,Esposizione esternaBalconeInfissi esterni in d...,https://pic.im-cdn.it/plan/99386393/m.jpg


In [5]:
DB_df

,Date_GMT,Titolo,Area,Link,Prezzo,Locali,MQ,Bagni,Piano,Descrizione,Dettagli,Agenzia,contratto,anno di costruzione,stato,riscaldamento,efficienza energetica,altre caratteristiche,Blueprint Url
0,2023-07-31 17:13:44 GMT,Trilocale viale Miramare 23,"Roiano, Trieste",https://www.immobiliare.it/annunci/104849015/,€ 270.000,3,135m²,1,3,PDM Real Estate propone in vendita questo spet...,PDM Real Estate propone in vendita questo spet...,SPUNTON & PARTNERS,Vendita,1905,Da ristrutturare,"Autonomo, a radiatori, alimentato a metano",F109.04 kWh/m² anno,Esposizione esternaBalconeImpianto tv singoloP...,https://pic.im-cdn.it/plan/99318099/m.jpg
1,2023-07-31 17:13:45 GMT,Bilocale Strada di Portopiccolo,Duino-Aurisina,https://www.immobiliare.it/annunci/98582788/,€ 489.000,2,84m²,1,1,NUOVO APPARTAMENTO - Nel prestigioso complesso...,NUOVO APPARTAMENTO - Nel prestigioso complesso...,PORTO PICCOLO,Vendita - Scarica capitolato,2014,Nuovo / In costruzione,"Centralizzato, a pavimento, alimentato a pompa...",A≥ 175 kWh/m² anno,Cancello elettricoFibra otticaPorta blindataEs...,https://pic.im-cdn.it/plan/99253501/m.jpg
2,2023-07-31 17:13:45 GMT,Appartamenti e Attici di nuova costruzione a T...,None,https://www.immobiliare.it/annunci/99036760/,€ 160.000 - € 500.000,1 - 4,da57m²,None,None,Le Terrazze sul Golfo è il nuovo progetto resi...,Le Terrazze sul Golfo è il nuovo progetto resi...,Terrazze sul Golfo,Vendita,NaN,NaN,"Centralizzato, a pavimento, alimentato a pompa...",NaN,Parcheggio biciGiardino comuneInfissi esterni ...,Not available
3,2023-07-31 17:13:44 GMT,Attico viale Miramare 109,"Barcola, Trieste",https://www.immobiliare.it/annunci/85478590/,€ 650.000,3,105m²,2,"2 piani: Seminterrato, 4°, con ascensore, con ...",Meraviglioso attico nel residence nel compless...,Meraviglioso attico nel residence nel compless...,Gruppo Greif,Vendita,2021,Nuovo / In costruzione,"Autonomo, a pavimento, alimentato a pompa di c...",A+≥ 175 kWh/m² anno,IdromassaggioCampo da tennisCancello elettrico...,Not available
4,2023-07-31 17:13:44 GMT,Trilocale viale Miramare 23,"Roiano, Trieste",https://www.immobiliare.it/annunci/104926247/,€ 270.000,3,135m²,1,3,Proponiamo in vendita questo spettacolare appa...,Proponiamo in vendita questo spettacolare appa...,STUDIO ROIANO S.R.L.,Vendita,1905,Da ristrutturare,"Autonomo, a radiatori, alimentato a gas",F109.04 kWh/m² anno,Esposizione esternaBalconeInfissi esterni in d...,https://pic.im-cdn.it/plan/99386393/m.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,2023-07-31 17:17:02 GMT,Appartamento piazza del Sansovino 2/1,"San Giacomo, Trieste",https://www.immobiliare.it/annunci/102984936/,€ 125.000,None,70m²,None,6,"appartamento anni 70, sesto piano, in buone co...","appartamento anni 70, sesto piano, in buone co...",None,Vendita,NaN,NaN,Centralizzato,F120 kWh/m² anno,BalconeParzialmente Arredato,https://pic.im-cdn.it/plan/97657904/m.jpg
1452,2023-07-31 17:17:02 GMT,Palazzo - Edificio in Vendita,None,https://www.immobiliare.it/annunci/90640867/,€ 1.000.000,None,1m²,None,60,"Ricerco, per pronto acquisto, immobile terra-c...","Ricerco, per pronto acquisto, immobile terra-c...",None,Vendita | Immobile a reddito,1800,Da ristrutturare,NaN,NaN,Esposizione esternaInfissi esterni in vetro / ...,Not available
1453,2023-07-31 17:17:02 GMT,Villa unifamiliare Località Basovizza 218,"Basovizza - Trebiciano, Trieste",https://www.immobiliare.it/annunci/103293662/,€ 400.000,None,800m²,None,"2 piani: Interrato (-1), Piano rialzato",Casa composta da cantina piano fuori terra e g...,Casa composta da cantina piano fuori terra e g...,None,Vendita,1955,Da ristrutturare,Autonomo,NaN,Giardino privatoInfissi esterni in vetro / met...,Not available
1454,2023-07-31 17:17:02 GMT,Appartamento frazione Duino 72,"Duino, Duino-Aurisina",https://www.immobiliare.it/annunci/104952063/,€ 430.000,5,140m²,2,1 - 2,"Appartamento di testa Duino140 mq, su due pian...","Appartamento di testa Duino140 mq, su due pian...",None,Vendita,